# Module

In [1]:
import numpy as np
import pandas as pd
import warnings
import gc
from tqdm import tqdm_notebook as tqdm
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore")
gc.enable()

In [2]:
pd.set_option('max_rows', 500)
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)

# Load Data

In [3]:
train_raw = pd.read_csv('./data/train.csv')
test_raw = pd.read_csv('./data/test.csv')
train_raw.shape, test_raw.shape

((200000, 202), (200000, 201))

In [4]:
train = train_raw.copy()
test = test_raw.copy()

In [6]:
train0 = train[ train['target']==0 ].copy()
train1 = train[ train['target']==1 ].copy()
train.sample(5)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
26959,train_26959,1,10.9020,-2.4469,8.3611,9.0721,11.1254,-20.7582,5.9289,23.0520,-1.7027,7.7488,7.2279,9.7263,13.9879,6.6539,6.7474,14.5837,8.9329,-15.3794,14.8353,13.0610,17.6761,5.3006,6.8549,2.7601,15.0941,13.4273,9.5878,-1.5014,6.4776,6.5432,3.3260,15.1735,1.8596,12.0803,11.1250,-5.2507,0.0273,2.8159,11.2346,-0.2105,-5.4246,0.8354,11.2028,11.0027,21.8670,-6.5235,13.7263,-4.7502,7.5214,2.9336,13.0326,31.1174,1.5176,6.0495,6.8654,16.7275,17.0697,6.5828,8.6548,9.5882,5.4915,-10.0686,2.8850,3.9191,4.3162,8.6379,5.1823,10.5702,5.0336,-12.3759,47.2380,0.4197,1.0019,22.8037,12.7163,20.2165,8.3896,13.4361,6.7104,12.9666,-2.0214,14.0614,-19.4478,-11.2834,-15.2922,16.2145,5.7705,15.6333,9.0827,5.0037,12.2056,7.0629,6.9441,10.5599,13.3730,0.4399,23.9972,35.0270,2.3498,-1.8746,-13.6670,16.7196,10.8343,1.6356,11.6453,2.8869,11.3984,29.9287,14.3255,10.4788,12.2098,5.2938,1.9017,8.6342,1.0938,0.5469,2.4565,41.7947,5.1012,7.2540,33.8180,11.3582,-9.1117,6.0430,7.2604,12.8605,12.9550,1.2358,0.3965,21.2929,13.9372,1.1065,9.5562,6.5202,2.0454,-12.4567,31.3434,13.5630,-0.7390,1.4853,3.0026,13.4852,7.0206,8.8630,7.9419,3.4923,9.0049,2.0253,4.1752,-2.9151,16.8085,5.1612,8.2249,19.9676,2.5991,3.6567,14.1400,-9.0241,33.0255,11.3018,31.5758,5.8212,5.8359,11.6741,-6.2794,26.2844,2.8498,-9.7853,11.5471,5.7151,-1.3780,3.6194,5.5033,0.2845,20.0954,8.4425,2.3138,9.1526,-4.5214,9.6166,-3.5431,10.4218,10.7097,7.1401,4.0658,-3.1097,10.8821,13.6953,10.9401,0.7766,6.1334,3.3935,0.7549,4.3573,11.6641,-0.3244,0.7191,7.2878,16.5992,-16.9856
173891,train_173891,0,13.3032,1.1754,13.2192,4.3670,12.0850,2.7608,6.0757,20.1818,4.1484,7.2188,5.6963,3.7118,14.0395,11.1978,10.1512,14.9258,10.6049,-13.4389,11.9389,19.0922,18.4121,24.3287,2.9952,2.8679,10.5971,13.1310,-11.8389,-3.8325,6.0759,7.3266,-6.3761,11.4195,-3.3733,19.4700,11.7235,10.1116,-1.9405,8.4649,11.5309,-2.4327,-7.9744,15.6393,12.7105,11.4638,16.4180,-3.5777,16.0417,-16.6211,9.4083,15.6793,11.1974,10.7947,0.6036,6.6430,-9.9924,9.2036,21.8421,6.6956,-5.0313,8.9595,21.9896,-12.9194,-1.4197,3.5659,4.1546,5.5033,7.6317,4.2737,5.0157,3.5613,6.8697,0.9265,4.2854,12.1615,5.2600,13.6117,11.7925,14.8217,7.2413,14.5759,7.1642,16.3678,3.9759,7.2153,7.8924,25.8948,3.1484,6.0144,8.4022,3.4543,-11.6191,7.0121,8.0093,11.2178,12.0815,0.2751,14.8108,25.5508,1.6796,0.8257,-14.6943,12.0771,32.6644,1.4250,14.2070,3.1395,8.4447,23.9606,14.1778,16.8255,5.6815,7.0013,4.5756,7.6290,1.6926,1.5316,4.3341,-0.4634,-8.7961,-0.6554,13.7371,10.9188,0.6110,6.23

In [7]:
col_list = train.columns[2:]

In [6]:
# CALCULATE MEANS AND STANDARD DEVIATIONS
s = [0]*200
m = [0]*200
for i in range(200):
    s[i] = np.std(train['var_'+str(i)])
    m[i] = np.mean(train['var_'+str(i)])
    
# CALCULATE PROB(TARGET=1 | X)
def getp(i,x):
    c = 3 #smoothing factor
    a = len( train1[ (train1['var_'+str(i)]>x-s[i]/c)&(train1['var_'+str(i)]<x+s[i]/c) ] ) 
    b = len( train0[ (train0['var_'+str(i)]>x-s[i]/c)&(train0['var_'+str(i)]<x+s[i]/c) ] )
    # if a+b<500: return 0.1 #smoothing factor
    # RETURN PROBABILITY
    return a / (a+b)
    # ALTERNATIVELY RETURN ODDS
    # return a / b
    
# SMOOTH A DISCRETE FUNCTION
def smooth(x,st=1):
    for j in range(st):
        x2 = np.ones(len(x)) * 0.1
        for i in range(len(x)-2):
            x2[i+1] = 0.25*x[i]+0.5*x[i+1]+0.25*x[i+2]
        x = x2.copy()
    return x

In [17]:
# DRAW PLOTS, YES OR NO
Picture = False
# DATA HAS Z-SCORE RANGE OF -4.5 TO 4.5
rmin=-5; rmax=5; 
# CALCULATE PROBABILITIES FOR 501 BINS
res=501
# STORE PROBABILITIES IN PR
pr = 0.1 * np.ones((200,res))
pr2 = pr.copy()
xr = np.zeros((200,res))
xr2 = xr.copy()
ct2 = 0
for j in tqdm(range(50)):
    if Picture: plt.figure(figsize=(15,8))
    for v in range(4):
        ct = 0
        # CALCULATE PROBABILITY FUNCTION FOR VAR
        for i in np.linspace(rmin,rmax,res):
            pr[v+4*j,ct] = getp(v+4*j,m[v+4*j]+i*s[v+4*j])
            xr[v+4*j,ct] = m[v+4*j]+i*s[v+4*j]
            xr2[v+4*j,ct] = i
            ct += 1
        if Picture:
            # SMOOTH FUNCTION FOR PRETTIER DISPLAY
            # BUT USE UNSMOOTHED FUNCTION FOR PREDICTION
            pr2[v+4*j,:] = smooth(pr[v+4*j,:],res//10)
            # DISPLAY PROBABILITY FUNCTION
            plt.subplot(2, 4, ct2%4+5)
            plt.plot(xr[v+4*j,:],pr2[v+4*j,:],'-')
            plt.title('P( t=1 | var_'+str(v+4*j)+' )')
            xx = plt.xlim()
            # DISPLAY TARGET DENSITIES
            plt.subplot(2, 4, ct2%4+1)            
            sns.distplot(train0['var_'+str(v+4*j)], label = 't=0')
            sns.distplot(train1['var_'+str(v+4*j)], label = 't=1')
            plt.title('var_'+str(v+4*j))
            plt.legend()
            plt.xlim(xx)
            plt.xlabel('')
        ct2 += 1
    if Picture: plt.show()

In [127]:
def get_pred(data):

    on = np.nonzero(xr[int(col.split('_')[-1])] > data)[0][0]
    pred = (pr[int(col[-1])][on] + pr[int(col[-1])][on + 1])/2

    return pred

In [131]:
prob_dic = {}

for col in tqdm(col_list):

    temp = train[col].value_counts().to_frame()

    temp['pred'] = temp.index.map(get_pred)
    temp = np.around(temp, 5)
    del temp[col]

    temp = temp.to_dict()['pred']
    
    prob_dic[col] = temp

In [166]:
pb_idx = np.load('./data_temp/public_LB.npy')
pv_idx = np.load('./data_temp/private_LB.npy')

In [167]:
test_pb = test.iloc[pb_idx].sort_index().copy()
test_pv = test.iloc[pv_idx].sort_index().copy()

test_real = test_pb.append(test_pv)

In [168]:
data = train.append(test_real)

In [138]:
unique_df = data[['ID_code']]

In [139]:
for col in tqdm(col_list):
    unique_df[col] = data[col].map(((data[col].value_counts() == 1) * 1).to_dict())

In [174]:
# unique_data = data[['ID_code', 'target']]
for col in tqdm(col_list):
    data[col + '_unique'] = np.around(data[col] * unique_df[col], 4)
#     unique_data[col] = np.around(data[col] * unique_df[col], 4)

In [176]:
for col in tqdm(col_list):
    data[col + '_prob'] = data[col + '_unique'].map(prob_dic[col])

In [179]:
train = data[~data.target.isna()]
test = data[data.target.isna()]

In [180]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [183]:
target = train['target']

In [184]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average': False,
    'boost': 'gbdt',
    'feature_fraction_seed': 47,
    'feature_fraction': 0.041,
    'learning_rate': 0.01,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1,
    'num_threads': 8
}

In [185]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
feature_importance = pd.DataFrame()

train_columns = [c for c in train.columns if c not in ['ID_code', 'target']]

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target.values)):    
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])

    num_round = 30000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=400, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    predictions_lgb += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits

    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = clf.feature_importance()
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
    print("CV score: {:<8.5f}".format(roc_auc_score(target.values[val_idx], oof_lgb[val_idx])))
    
print("CV score: {:<8.5f}".format(roc_auc_score(target.values, oof_lgb)))

fold n°0
Training until validation scores don't improve for 200 rounds.
[400]	training's auc: 0.890433	valid_1's auc: 0.872906
[800]	training's auc: 0.901113	valid_1's auc: 0.882236
[1200]	training's auc: 0.907121	valid_1's auc: 0.886424
[1600]	training's auc: 0.912618	valid_1's auc: 0.890582
[2000]	training's auc: 0.916292	valid_1's auc: 0.892593
[2400]	training's auc: 0.919965	valid_1's auc: 0.894464
[2800]	training's auc: 0.922948	valid_1's auc: 0.895916
[3200]	training's auc: 0.925863	valid_1's auc: 0.897298
[3600]	training's auc: 0.9286	valid_1's auc: 0.898492
[4000]	training's auc: 0.931252	valid_1's auc: 0.89941
[4400]	training's auc: 0.933689	valid_1's auc: 0.900249
[4800]	training's auc: 0.936123	valid_1's auc: 0.901017
[5200]	training's auc: 0.93854	valid_1's auc: 0.902014
[5600]	training's auc: 0.940774	valid_1's auc: 0.902838
[6000]	training's auc: 0.942989	valid_1's auc: 0.903607
[6400]	training's auc: 0.945074	valid_1's auc: 0.904235
[6800]	training's auc: 0.947184	valid_